In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
import pandas as pd
df = pd.read_csv('flight data.csv', on_bad_lines='skip')
df.head(5)

,from_airport_code,from_country,dest_airport_code,dest_country,aircraft_type,airline_number,airline_name,flight_number,departure_time,arrival_time,duration,stops,price,currency,co2_emissions,avg_co2_emission_for_this_route,co2_percentage,scan_date
0,ALG,Algeria,AEP,Argentina,Airbus A318|Canadair RJ 1000|Airbus A330|Airbu...,multi,[Air France| Iberia| LATAM],AF1491|AF1491|AF1491|AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1279.0,USD,1320000.0,1320000.0,0%,2022-04-29 17:52:59
1,ALG,Algeria,AEP,Argentina,Airbus A318|Canadair RJ 1000|Boeing 787|Airbus...,multi,[Air France| Iberia| LATAM],AF1491|AF1491|AF1491|AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1279.0,USD,1195000.0,1320000.0,-9%,2022-04-29 17:52:59
2,ALG,Algeria,AEP,Argentina,Airbus A320|Airbus A321|Boeing 787|Airbus A320,multi,[Air France| LATAM],AF1855|AF1855|AF1855|AF1855,2022-04-30 12:45:00,2022-05-01 10:15:00,1530,3,1284.0,USD,1248000.0,1320000.0,-5%,2022-04-29 17:52:59
3,ALG,Algeria,AEP,Argentina,Airbus A318|Airbus A320|Boeing 787|Airbus A320,multi,[Air France| LATAM],AF1491|AF1491|AF1491|AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1290.0,USD,1347000.0,1320000.0,2%,2022-04-29 17:52:59
4,ALG,Algeria,AEP,Argentina,Airbus A321neo|Boeing 777|Airbus A320,multi,[Lufthansa| LATAM],LH1317|LH1317|LH1317,2022-04-30 12:35:00,2022-05-01 10:15:00,1540,2,1347.0,USD,1381000.0,1320000.0,4%,2022-04-29 17:52:59


In [3]:
# Drop duplicate values
df.drop_duplicates(inplace=True)

# Drop rows with missing values
df.dropna(inplace=True)

In [4]:
# remove the airline_name column in  pandas DataFrame that contains square brackets []

df['airline_name'] = df['airline_name'].str.replace('[','').str.replace(']','')
df['airline_name'] = df['airline_name'].str.split('|').str[0]

/var/folders/t3/ctmyp9cd5j7ctjhqp2_g843h0000gn/T/ipykernel_3932/555518936.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['airline_name'] = df['airline_name'].str.replace('[','').str.replace(']','')


In [5]:
# delete the duplicate flight number in column flight_number behind the strings with a | separator

df['flight_number'] = df['flight_number'].str.split('|').str[0]

In [6]:
# Get the datetime info

df['departure_month'] = pd.to_datetime(df['departure_time']).dt.month

In [7]:
# Check Whether the units are standardized

df['currency'].unique()

array(['USD'], dtype=object)

In [8]:
df['co2_percentage'] = df['co2_percentage'].str.replace('%','')
df.head()

,from_airport_code,from_country,dest_airport_code,dest_country,aircraft_type,airline_number,airline_name,flight_number,departure_time,arrival_time,duration,stops,price,currency,co2_emissions,avg_co2_emission_for_this_route,co2_percentage,scan_date,departure_month
0,ALG,Algeria,AEP,Argentina,Airbus A318|Canadair RJ 1000|Airbus A330|Airbu...,multi,Air France,AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1279.0,USD,1320000.0,1320000.0,0,2022-04-29 17:52:59,4
1,ALG,Algeria,AEP,Argentina,Airbus A318|Canadair RJ 1000|Boeing 787|Airbus...,multi,Air France,AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1279.0,USD,1195000.0,1320000.0,-9,2022-04-29 17:52:59,4
2,ALG,Algeria,AEP,Argentina,Airbus A320|Airbus A321|Boeing 787|Airbus A320,multi,Air France,AF1855,2022-04-30 12:45:00,2022-05-01 10:15:00,1530,3,1284.0,USD,1248000.0,1320000.0,-5,2022-04-29 17:52:59,4
3,ALG,Algeria,AEP,Argentina,Airbus A318|Airbus A320|Boeing 787|Airbus A320,multi,Air France,AF1491,2022-04-30 14:30:00,2022-05-01 10:15:00,1425,3,1290.0,USD,1347000.0,1320000.0,2,2022-04-29 17:52:59,4
4,ALG,Algeria,AEP,Argentina,Airbus A321neo|Boeing 777|Airbus A320,multi,Lufthansa,LH1317,2022-04-30 12:35:00,2022-05-01 10:15:00,1540,2,1347.0,USD,1381000.0,1320000.0,4,2022-04-29 17:52:59,4


In [9]:
# Change object variables to category type

for col in df.columns:
    # Check if the column is of object type
    if df[col].dtype == 'object':
        # If it is, convert it to category type
        df[col] = df[col].astype('category')

In [10]:
df['co2_percentage'] = df['co2_percentage'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847355 entries, 0 to 998865
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   from_airport_code                847355 non-null  category
 1   from_country                     847355 non-null  category
 2   dest_airport_code                847355 non-null  category
 3   dest_country                     847355 non-null  category
 4   aircraft_type                    847355 non-null  category
 5   airline_number                   847355 non-null  category
 6   airline_name                     847355 non-null  category
 7   flight_number                    847355 non-null  category
 8   departure_time                   847355 non-null  category
 9   arrival_time                     847355 non-null  category
 10  duration                         847355 non-null  int64   
 11  stops                            847355 non-null  in

In [11]:
from_country_mapping = dict(enumerate(df['from_country'].cat.categories))
print(from_country_mapping)

{0: 'Algeria', 1: 'Argentina', 2: 'Australia', 3: 'Austria', 4: 'Belgium', 5: 'Brazil', 6: 'Canada', 7: 'Chile', 8: 'China', 9: 'Columbia', 10: 'Denmark', 11: 'Dublin', 12: 'Egypt', 13: 'Ethiopia', 14: 'France', 15: 'Germany', 16: 'Greece', 17: 'India'}


In [12]:
dest_country_mapping = dict(enumerate(df['dest_country'].cat.categories))
print(dest_country_mapping)

{0: 'Algeria', 1: 'Argentina', 2: 'Australia', 3: 'Austria', 4: 'Belgium', 5: 'Brazil', 6: 'Canada', 7: 'Chile', 8: 'China', 9: 'Columbia', 10: 'Denmark', 11: 'Dublin', 12: 'Egypt', 13: 'Ethiopia', 14: 'France', 15: 'Germany', 16: 'Greece', 17: 'India', 18: 'Indonesia', 19: 'Italy', 20: 'Japan', 21: 'Kenya', 22: 'Malaysia', 23: 'Mexico', 24: 'Morocco', 25: 'Netherlands', 26: 'Norway', 27: 'Panama', 28: 'Peru', 29: 'Philippines', 30: 'Portugal', 31: 'Qatar', 32: 'Rome', 33: 'Russia', 34: 'Singapore', 35: 'South Africa', 36: 'South Korea', 37: 'Spain', 38: 'Sweden', 39: 'Taiwan', 40: 'Thailand', 41: 'Turkey', 42: 'United Arab Emirates', 43: 'United Kingdom', 44: 'United States', 45: 'Vietnam', 46: 'Zurich'}


In [13]:
airline_name_mapping = dict(enumerate(df['airline_name'].cat.categories))
print(airline_name_mapping)

{0: 'ANA', 1: 'ASL Airlines', 2: 'Aegean', 3: 'Aer Lingus', 4: 'Aerolineas Argentinas', 5: 'Aeromexico', 6: 'Air Algerie', 7: 'Air Arabia', 8: 'Air Arabia Maroc', 9: 'Air Astana', 10: 'Air Austral', 11: 'Air Baltic', 12: 'Air Canada', 13: 'Air China', 14: 'Air Dolomiti', 15: 'Air Europa', 16: 'Air France', 17: 'Air India', 18: 'Air Macau', 19: 'Air Malta', 20: 'Air Mauritius', 21: 'Air Moldova', 22: 'Air New Zealand', 23: 'Air Niugini', 24: 'Air Serbia', 25: 'Air Seychelles', 26: 'Air Tahiti Nui', 27: 'Air Transat', 28: 'Air-India Express', 29: 'AirAsia (India)', 30: 'AirAsia X', 31: 'Aircalin', 32: 'American', 33: 'Arkia', 34: 'Asiana', 35: 'Austrian', 36: 'Avianca', 37: 'Azores Airlines', 38: 'Azul', 39: 'Bamboo Airways', 40: 'Biman', 41: 'Blue Air', 42: 'BoA', 43: 'British Airways', 44: 'Brussels Airlines', 45: 'Bulgaria Air', 46: 'COPA', 47: 'CSA', 48: 'Cathay Pacific', 49: 'Cebu Pacific', 50: 'China Airlines', 51: 'China Eastern', 52: 'China Southern', 53: 'Corendon', 54: 'Croatia

In [14]:
# Convert text data to numbers

df['from_country'] = df['from_country'].cat.codes
df['dest_country'] = df['dest_country'].cat.codes
df['airline_name'] = df['airline_name'].cat.codes

In [15]:
# Define the X columns and Y columns

X = df[['from_country','dest_country','airline_name','duration','stops','co2_emissions','departure_month']]
Y = df[['price']]

In [16]:
# Standardization

sc = StandardScaler()
X = sc.fit_transform(X)

In [17]:
# Split the dataset into training data and testing data

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.2, random_state = 42)

## Decision Tree Model

In [18]:
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor().fit(Xtrain, Ytrain)
Ypred = DTR.predict(Xtest)

In [19]:
# Evaluate the model's performance
r_squared = r2_score(Ytest, Ypred)
mse = mean_squared_error(Ytest, Ypred)
mae = mean_absolute_error(Ytest, Ypred)

# Print the evaluation metrics
print(f"R-squared: {r_squared:.2f}")
print(f"Mean squared error: {mse:.2f}")
print(f"Mean absolute error: {mae:.2f}")

R-squared: 0.80
Mean squared error: 684344.04
Mean absolute error: 325.93


In [20]:
dump(DTR, 'DecisionTree.joblib')

['DecisionTree.joblib']

## KNN Model

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

KN = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}
grid_search = GridSearchCV(KN, param_grid, cv=5, scoring='accuracy')
grid_search.fit(Xtrain, Ytrain)

print("Best Hyperparameters:", grid_search.best_params_)
print("Accuracy Score:", grid_search.best_score_)

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expect

Best Hyperparameters: {'n_neighbors': 3}
Accuracy Score: 0.18578990115176922


In [19]:
KN = KNeighborsClassifier(n_neighbors = 3).fit(Xtrain, Ytrain)
Ypred = KN.predict(Xtest)

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [20]:
# Evaluate the model's performance
r_squared = r2_score(Ytest, Ypred)
mse = mean_squared_error(Ytest, Ypred)
mae = mean_absolute_error(Ytest, Ypred)

# Print the evaluation metrics
print(f"R-squared: {r_squared:.2f}")
print(f"Mean squared error: {mse:.2f}")
print(f"Mean absolute error: {mae:.2f}")

R-squared: 0.71
Mean squared error: 983826.29
Mean absolute error: 420.91


In [21]:
dump(KN, 'KNN.joblib')

['KNN.joblib']

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10, random_state=42).fit(Xtrain, Ytrain)
Ypred = rf.predict(Xtest)

/var/folders/t3/ctmyp9cd5j7ctjhqp2_g843h0000gn/T/ipykernel_3781/1439846218.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = RandomForestClassifier(n_estimators=10, random_state=42).fit(Xtrain, Ytrain)


In [ ]:
# Evaluate the model's performance
r_squared = r2_score(Ytest, Ypred)
mse = mean_squared_error(Ytest, Ypred)
mae = mean_absolute_error(Ytest, Ypred)

# Print the evaluation metrics
print(f"R-squared: {r_squared:.2f}")
print(f"Mean squared error: {mse:.2f}")
print(f"Mean absolute error: {mae:.2f}")